In [ ]:
from keras.models import load_model
new_model = load_model('drowsiness_model.h5')

In [ ]:
new_model.summary()
new_model.get_weights()
new_model.optimizer

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [ ]:
test_path = r"C:\Users\Hayavadana Sadana\Desktop\AAPS\Images\mrlEyes_2018_01\test"
test_batches = ImageDataGenerator().flow_from_directory(test_path,target_size=(25,25),classes=['open','close'],batch_size=2142)

In [ ]:
predictions = new_model.predict_generator(test_batches,steps=2.142,verbose=0)

In [ ]:
predictions

In [ ]:
for i in predictions:
    if i[0]>i[1]:
        i[0]= 1
        i[1]=0
    else:
        i[1]=1
        i[0]=0

In [ ]:
predictions

### check individual images and their predictions

In [ ]:
test1 = test_batches[0][0][241]
imgh = plt.imshow(test1[:, :, 0])
test1 = np.array(test1)
test1 = test1.reshape(1,25,25,3)
print(new_model.predict(test1))
print("open") if new_model.predict(test1)[0][0] > new_model.predict(test1)[0][1]else print("close")

### Camera module code // prints number of closed eyes predicted

In [ ]:
import subprocess
import cv2
eye_cascade=cv2.CascadeClassifier('haarcascade_eye-Copy1.xml')
face_cascade=cv2.CascadeClassifier('haarcascade_frontalface_default-Copy1.xml')
def checkme(l):

    che = l[0]
    if che[0]>che[1]:
        
        return 1
    else:
        
        return 0

    
cap=cv2.VideoCapture(0)
try:
    while True:
        no_of_eyes=0
        closed_no=0
        done=False
        while no_of_eyes<25:
            ret,img=cap.read()
            gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
            
            faces=face_cascade.detectMultiScale(gray,1.3,5)
            
            for (x,y,w,h) in faces:
                cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
                roi_gray=gray[y:y+h,x:x+h]
                roi_img=img[y:y+h,x:x+h]
                eyes=eye_cascade.detectMultiScale(roi_gray)
                for (ex,ey,ew,eh) in eyes:
                    
                    
                    no_of_eyes+=1
                    want_img=roi_img[ey:ey+eh,ex:ex+ew]
                    want_img=cv2.resize(want_img,(25,25))
#                     want_img=want_img/255
                    want_img = want_img.reshape(1,25,25,3)
#                     want_img = np.array(want_img)
#                     want_img = want_img/255
#                     want_img = want_img.reshape(1,25,25,3)
#                     print(checkme(new_model.predict(want_img)))
                    closed_no+=1 if checkme(new_model.predict(want_img))==0  else 0
                    
            cv2.imshow('image',img)
            k=cv2.waitKey(32) & 0xff
            if k==27:
                done=True
                cv2.destroyAllWindows()
                break
        print(closed_no)
        if closed_no>9:
            subprocess.call("cscript helloi-Copy1.vbs")
        if done:
            cap.release()
            break
except KeyboardInterrupt:
    cv2.destroyAllWindows()

    cap.release()